### Задание 1 - Николаев
Попытка выполнить в PyTorch, ибо нефиг.

In [1]:
import torch
import numpy as np
import os 


from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from sklearn.metrics import accuracy_score

from keras.datasets import imdb

Using TensorFlow backend.


In [2]:
dtype = torch.cuda.FloatTensor # для переноса нагрузки на GPU

In [3]:
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

In [4]:
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000) 

#### (explanation from a website): Data loading in PyTorch is in 2 parts:

First the data must be wrapped in a Dataset class with a getitem method that from an index return X_train[index] and y_train[index] and a length method. A Dataset is basically a data storage.

In [5]:
class IMDBDatasetTrain(Dataset):
    def __init__(self, data, labels):
        self.X_train = vectorize_sequences(data)
        self.y_train = np.asarray(labels).astype('float32')
    def __getitem__(self, index):
        item = self.X_train[index]
        label = self.y_train[index]
        return item, label
    def __len__(self):
        return len(self.X_train)

In [6]:
dset_train = IMDBDatasetTrain(train_data, train_labels)

Second part: we provided PyTorch with a data storage, and we have to tell it how to load it. This is done with DataLoader.

The DataLoader defines how you retrieve the items + labels from the dataset. You can tell it to:

Set the batch size.
Shuffle and sample the data randomly, hence implementing train_test_split (as in SubsetRandomSampler)

In [7]:
train_loader = DataLoader(dset_train,
                          batch_size = 512,
                          shuffle = True,
                          num_workers = 0, # 4 for CUDA, 1 for Ubuntu, 0 for Windows
                          pin_memory = True)  # unspecified for CPU, specified for CUDA

### Бинарная сетка

In [8]:
epochs = 20

In [9]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(10000, 16)
        self.fc2 = nn.Linear(16, 1)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.sigmoid(x)
    
model = Net().cuda() # for GPU

In [10]:
optimizer = optim.RMSprop(model.parameters(), lr=0.001)

In [11]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).type(dtype), Variable(target).type(dtype)
        optimizer.zero_grad()
        output = model(data)
        loss = F.binary_cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 2 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch+1, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

In [12]:
for epoch in range(epochs):
    train(epoch)

c:\programdata\miniconda3\lib\site-packages\torch\nn\functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
c:\programdata\miniconda3\lib\site-packages\torch\nn\functional.py:1594: UserWarning: Using a target size (torch.Size([512])) that is different to the input size (torch.Size([512, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))
c:\programdata\miniconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  del sys.path[0]


Train Epoch: 1 [0/25000 (0%)]	Loss: 0.704275
Train Epoch: 1 [1024/25000 (4%)]	Loss: 0.693540
Train Epoch: 1 [2048/25000 (8%)]	Loss: 0.650261
Train Epoch: 1 [3072/25000 (12%)]	Loss: 0.621235
Train Epoch: 1 [4096/25000 (16%)]	Loss: 0.593387
Train Epoch: 1 [5120/25000 (20%)]	Loss: 0.575729
Train Epoch: 1 [6144/25000 (24%)]	Loss: 0.564508
Train Epoch: 1 [7168/25000 (29%)]	Loss: 0.563516
Train Epoch: 1 [8192/25000 (33%)]	Loss: 0.547607
Train Epoch: 1 [9216/25000 (37%)]	Loss: 0.547469
Train Epoch: 1 [10240/25000 (41%)]	Loss: 0.487480
Train Epoch: 1 [11264/25000 (45%)]	Loss: 0.525624
Train Epoch: 1 [12288/25000 (49%)]	Loss: 0.484419
Train Epoch: 1 [13312/25000 (53%)]	Loss: 0.496202
Train Epoch: 1 [14336/25000 (57%)]	Loss: 0.487536
Train Epoch: 1 [15360/25000 (61%)]	Loss: 0.476722
Train Epoch: 1 [16384/25000 (65%)]	Loss: 0.494039
Train Epoch: 1 [17408/25000 (69%)]	Loss: 0.454751
Train Epoch: 1 [18432/25000 (73%)]	Loss: 0.458144
Train Epoch: 1 [19456/25000 (78%)]	Loss: 0.450416
Train Epoch: 1 [

c:\programdata\miniconda3\lib\site-packages\torch\nn\functional.py:1594: UserWarning: Using a target size (torch.Size([424])) that is different to the input size (torch.Size([424, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Train Epoch: 1 [20352/25000 (98%)]	Loss: 0.442669
Train Epoch: 2 [0/25000 (0%)]	Loss: 0.440060
Train Epoch: 2 [1024/25000 (4%)]	Loss: 0.411390
Train Epoch: 2 [2048/25000 (8%)]	Loss: 0.396730
Train Epoch: 2 [3072/25000 (12%)]	Loss: 0.406964
Train Epoch: 2 [4096/25000 (16%)]	Loss: 0.420761
Train Epoch: 2 [5120/25000 (20%)]	Loss: 0.386928
Train Epoch: 2 [6144/25000 (24%)]	Loss: 0.378594
Train Epoch: 2 [7168/25000 (29%)]	Loss: 0.381667
Train Epoch: 2 [8192/25000 (33%)]	Loss: 0.387508
Train Epoch: 2 [9216/25000 (37%)]	Loss: 0.381449
Train Epoch: 2 [10240/25000 (41%)]	Loss: 0.430474
Train Epoch: 2 [11264/25000 (45%)]	Loss: 0.394094
Train Epoch: 2 [12288/25000 (49%)]	Loss: 0.395607
Train Epoch: 2 [13312/25000 (53%)]	Loss: 0.397236
Train Epoch: 2 [14336/25000 (57%)]	Loss: 0.372592
Train Epoch: 2 [15360/25000 (61%)]	Loss: 0.381235
Train Epoch: 2 [16384/25000 (65%)]	Loss: 0.388856
Train Epoch: 2 [17408/25000 (69%)]	Loss: 0.362407
Train Epoch: 2 [18432/25000 (73%)]	Loss: 0.373532
Train Epoch: 2 [

Train Epoch: 8 [15360/25000 (61%)]	Loss: 0.184613
Train Epoch: 8 [16384/25000 (65%)]	Loss: 0.207427
Train Epoch: 8 [17408/25000 (69%)]	Loss: 0.219065
Train Epoch: 8 [18432/25000 (73%)]	Loss: 0.190829
Train Epoch: 8 [19456/25000 (78%)]	Loss: 0.169501
Train Epoch: 8 [20480/25000 (82%)]	Loss: 0.179655
Train Epoch: 8 [21504/25000 (86%)]	Loss: 0.184610
Train Epoch: 8 [22528/25000 (90%)]	Loss: 0.229119
Train Epoch: 8 [23552/25000 (94%)]	Loss: 0.216136
Train Epoch: 8 [20352/25000 (98%)]	Loss: 0.182454
Train Epoch: 9 [0/25000 (0%)]	Loss: 0.162590
Train Epoch: 9 [1024/25000 (4%)]	Loss: 0.180806
Train Epoch: 9 [2048/25000 (8%)]	Loss: 0.157008
Train Epoch: 9 [3072/25000 (12%)]	Loss: 0.218022
Train Epoch: 9 [4096/25000 (16%)]	Loss: 0.196235
Train Epoch: 9 [5120/25000 (20%)]	Loss: 0.170829
Train Epoch: 9 [6144/25000 (24%)]	Loss: 0.188421
Train Epoch: 9 [7168/25000 (29%)]	Loss: 0.157828
Train Epoch: 9 [8192/25000 (33%)]	Loss: 0.179668
Train Epoch: 9 [9216/25000 (37%)]	Loss: 0.176962
Train Epoch: 9 [

Train Epoch: 15 [3072/25000 (12%)]	Loss: 0.114743
Train Epoch: 15 [4096/25000 (16%)]	Loss: 0.145561
Train Epoch: 15 [5120/25000 (20%)]	Loss: 0.110855
Train Epoch: 15 [6144/25000 (24%)]	Loss: 0.112541
Train Epoch: 15 [7168/25000 (29%)]	Loss: 0.110916
Train Epoch: 15 [8192/25000 (33%)]	Loss: 0.099931
Train Epoch: 15 [9216/25000 (37%)]	Loss: 0.124577
Train Epoch: 15 [10240/25000 (41%)]	Loss: 0.112432
Train Epoch: 15 [11264/25000 (45%)]	Loss: 0.130316
Train Epoch: 15 [12288/25000 (49%)]	Loss: 0.112499
Train Epoch: 15 [13312/25000 (53%)]	Loss: 0.116408
Train Epoch: 15 [14336/25000 (57%)]	Loss: 0.091822
Train Epoch: 15 [15360/25000 (61%)]	Loss: 0.107791
Train Epoch: 15 [16384/25000 (65%)]	Loss: 0.094705
Train Epoch: 15 [17408/25000 (69%)]	Loss: 0.112029
Train Epoch: 15 [18432/25000 (73%)]	Loss: 0.123025
Train Epoch: 15 [19456/25000 (78%)]	Loss: 0.099876
Train Epoch: 15 [20480/25000 (82%)]	Loss: 0.106074
Train Epoch: 15 [21504/25000 (86%)]	Loss: 0.147845
Train Epoch: 15 [22528/25000 (90%)]	Lo

## 2:40 ночи, начал в 20:00 : мне, наконец, удалось понять, откуда на Windows Memory Error, как перенести всё на GPU и в какой нормальной форме работать с PyTorch (потому что вариантов миллиард, а адекватный только этот). Окошко снизу для референса - показатели по одной из первых попыток (специально их здесь оставлю).

### 3:30 ночи, при очередном рестарте ядра для очистки памяти окошко я случайно грохнул. Простите.

In [13]:
from sklearn.metrics import accuracy_score

In [14]:
x, y = Variable(torch.from_numpy(vectorize_sequences(test_data))), Variable(torch.from_numpy(np.asarray(test_labels).astype('float32')), requires_grad=False)
pred = model(x.type(dtype))

c:\programdata\miniconda3\lib\site-packages\torch\nn\functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [16]:
final_pred = np.argmax(pred.data.cpu().numpy(), axis=1)

In [17]:
accuracy_score(np.asarray(test_labels), final_pred)

0.5

## Точность оставляет желать лучшего. Возможно, связано с тем, что я не разобрался, как сделать два линейных слоя: понятно для CNN, но для Linear не получается. Но это поправимо. И - учитывая все безумные трудности - оно оно хотя бы работает!

## Приступаем к многоклассовой классификации. Буду делать её в этом же файле, но, если куда-то не так нажму, у меня начнётся истерика: для того, чтобы сделать здесь же, нужно перезагружать ядро для очистки видеопамяти - при неудачных попытках делая это ОЧЕНЬ часто - и можно случайно убить все output'ы Jupyter'а.

In [1]:
import torch
import numpy as np
import os 


from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from sklearn.metrics import accuracy_score
from keras.utils.np_utils import to_categorical

from keras.datasets import reuters

Using TensorFlow backend.


In [2]:
dtype = torch.cuda.FloatTensor

In [3]:
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)

In [4]:
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

In [5]:
class ReutersDatasetTrain(Dataset):
    def __init__(self, data, labels):
        self.X_train = vectorize_sequences(data)
        self.y_train = to_categorical(labels)
    def __getitem__(self, index):
        item = self.X_train[index]
        label = self.y_train[index]
        return item, label
    def __len__(self):
        return len(self.X_train)

In [6]:
dset_train = ReutersDatasetTrain(train_data, train_labels)

In [7]:
train_loader = DataLoader(dset_train,
                          batch_size = 512,
                          shuffle = True,
                          num_workers = 0,
                          pin_memory = True)

In [8]:
epochs = 20

### Построим и обучим сеть.

In [24]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(10000, 64)
        self.fc2 = nn.Linear(64,64)
        self.fc3 = nn.Linear(64,46)
    def forward(self, x):
        x = F.tanh(self.fc1(x))
        x = F.dropout(x, 0.2)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, 0.2)
        x = self.fc3(x)
        return F.softmax(x)
    
model = Net().cuda()

In [25]:
optimizer = optim.RMSprop(model.parameters(), lr=0.001)

In [26]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).type(dtype), Variable(target).type(dtype)
        optimizer.zero_grad()
        output = model(data)
        loss = F.binary_cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 2 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch+1, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

In [27]:
for epoch in range(epochs):
    train(epoch)

c:\programdata\miniconda3\lib\site-packages\torch\nn\functional.py:995: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
c:\programdata\miniconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]
c:\programdata\miniconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  del sys.path[0]


Train Epoch: 1 [0/8982 (0%)]	Loss: 0.105386
Train Epoch: 1 [1024/8982 (11%)]	Loss: 0.087039
Train Epoch: 1 [2048/8982 (22%)]	Loss: 0.067745
Train Epoch: 1 [3072/8982 (33%)]	Loss: 0.056700
Train Epoch: 1 [4096/8982 (44%)]	Loss: 0.050416
Train Epoch: 1 [5120/8982 (56%)]	Loss: 0.048322
Train Epoch: 1 [6144/8982 (67%)]	Loss: 0.045251
Train Epoch: 1 [7168/8982 (78%)]	Loss: 0.044747
Train Epoch: 1 [8192/8982 (89%)]	Loss: 0.039976
Train Epoch: 2 [0/8982 (0%)]	Loss: 0.035131
Train Epoch: 2 [1024/8982 (11%)]	Loss: 0.038614
Train Epoch: 2 [2048/8982 (22%)]	Loss: 0.037272
Train Epoch: 2 [3072/8982 (33%)]	Loss: 0.033387
Train Epoch: 2 [4096/8982 (44%)]	Loss: 0.033156
Train Epoch: 2 [5120/8982 (56%)]	Loss: 0.034456
Train Epoch: 2 [6144/8982 (67%)]	Loss: 0.034082
Train Epoch: 2 [7168/8982 (78%)]	Loss: 0.033741
Train Epoch: 2 [8192/8982 (89%)]	Loss: 0.031818
Train Epoch: 3 [0/8982 (0%)]	Loss: 0.026049
Train Epoch: 3 [1024/8982 (11%)]	Loss: 0.027091
Train Epoch: 3 [2048/8982 (22%)]	Loss: 0.030835
Trai

Train Epoch: 19 [8192/8982 (89%)]	Loss: 0.003431
Train Epoch: 20 [0/8982 (0%)]	Loss: 0.005509
Train Epoch: 20 [1024/8982 (11%)]	Loss: 0.004932
Train Epoch: 20 [2048/8982 (22%)]	Loss: 0.003663
Train Epoch: 20 [3072/8982 (33%)]	Loss: 0.003058
Train Epoch: 20 [4096/8982 (44%)]	Loss: 0.003761
Train Epoch: 20 [5120/8982 (56%)]	Loss: 0.003590
Train Epoch: 20 [6144/8982 (67%)]	Loss: 0.004769
Train Epoch: 20 [7168/8982 (78%)]	Loss: 0.004531
Train Epoch: 20 [8192/8982 (89%)]	Loss: 0.004113


# Let's validate

In [28]:
from sklearn.metrics import accuracy_score

In [29]:
x, y = Variable(torch.from_numpy(vectorize_sequences(test_data))), Variable(torch.from_numpy(np.asarray(test_labels).astype('float32')), requires_grad=False)
pred = model(x.type(dtype))

c:\programdata\miniconda3\lib\site-packages\torch\nn\functional.py:995: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
c:\programdata\miniconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]


In [30]:
final_pred = np.argmax(pred.data.cpu().numpy(), axis=1)
accuracy_score(np.asarray(test_labels), final_pred)

0.7960819234194123

# Regression, yay !

In [1]:
import torch
import numpy as np
import os 

from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from sklearn.metrics import accuracy_score

from keras.datasets import boston_housing

Using TensorFlow backend.


In [2]:
dtype = torch.cuda.FloatTensor

In [3]:
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

In [4]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std

## Построим модель и отчаянно попытаемся внедрить K-fold

In [30]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(train_data.shape[1], 64)
        self.fc2 = nn.Linear(64,1)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        return self.fc2(x)
    
model = Net().cuda()

In [31]:
optimizer = torch.optim.RMSprop(model.parameters(), lr = 0.01)
criterion = nn.MSELoss()# Mean Squared Loss

In [32]:
def train(epoch, loader):
    model.train()
    for batch_idx, (data, target) in enumerate(loader):
        data, target = Variable(data).type(dtype), Variable(target).type(dtype)
        optimizer.zero_grad()
        output = model.forward(data)
        if target.dim() != output.dim():
            output = output.squeeze()
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

In [33]:
class HousingDatasetTrain(Dataset):
    def __init__(self, data, labels):
        self.X_train = data
        self.y_train = labels
    def __getitem__(self, index):
        item = self.X_train[index]
        label = self.y_train[index]
        return item, label
    def __len__(self):
        return len(self.X_train)

In [49]:
k = 4
num_val_samples = len(train_data) // k
num_epochs = 10
all_scores = []

In [56]:
for i in range(k):
    print('processing fold #', i)
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)
    
    dset_train = HousingDatasetTrain(partial_train_data, partial_train_targets)
    train_loader = DataLoader(dset_train,
                          batch_size = 1,
                          shuffle = False,
                          num_workers = 0,
                          pin_memory = True)
    
    
    for epoch in range(num_epochs):
        train(epoch, train_loader)
    predicted = model.forward(Variable(torch.from_numpy(val_data).type(dtype))).cpu().data.numpy()
    pred = np.swapaxes(predicted, 1, 0)
    sub = np.subtract(val_targets, pred)
    formae = np.absolute(sub[0])
    mae = np.sum(formae)/formae.size
    all_scores.append(mae)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3


In [57]:
all_scores

[2.6070554766324485,
 2.275290794183712,
 2.338240768885848,
 2.262471595613083,
 2.250446368680142]

In [58]:
np.mean(all_scores)

2.346701000799047

### На десяти эпохах (взятых для экономии времени) в итоге достигнут лучший результат, чем на 100, и даже на 500 эпохах в оригинале. За сим (ибо 6 часов утра :D) и откланиваюсь.